In [1]:
import pandas as pd

import pprint
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from sklearn.model_selection import train_test_split

from langchain_openai import ChatOpenAI

In [2]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='../Paper1-RAGG/impl/.env')

### OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import pickle
import torch
import os

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /home/hamzicd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Load and Test FewShotTTP (TTPFShot)

In [6]:
### embedding
model2 = "ehsanaghaei/SecureBERT"
embeddings2 = HuggingFaceEmbeddings(model_name = model2)

/tmp/ipykernel_3800975/3801564067.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings2 = HuggingFaceEmbeddings(model_name = model2)
No sentence-transformers model found with name ehsanaghaei/SecureBERT. Creating a new one with mean pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at ehsanaghaei/SecureBERT and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/hamzicd/.local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarnin

In [7]:
db = FAISS.load_local("./TTPFShot-training-data-19747", embeddings2, allow_dangerous_deserialization=True)

## TTPFShot - Classification Approach

In [111]:
### Predicting single TTP label
def predict_label(text_to_predict, train_vector_db, llm):
    ##### Prompt 1
    search_results = train_vector_db.similarity_search_with_score(text_to_predict, k=65)

    
    xs = ["\nText:"+ doc.page_content.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results if score < 2]
    #xs = ["\nText:'"+ doc.page_content.lower().strip()+"'\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results]
    

    xs = ''.join(xs)+"\nInputText:"+ text_to_predict.lower().strip() +"\nClassification:"
    xs = """You are an expert in MITRE ATT&CK TTP classification.
    Your task is to classify the 'InputText' based on the provided examples below.
    Each example shows a sentence, inverse similarity to InputText (the lower the better),
    and its corresponding classification label starting with 'T'.
    Use these examples to determine the correct classification for the given text (InputText).
    If 'InputText' is completely not related to any of the provided examples, return class 'T0000'.
    To determine if the 'InputText' is not related to the given examples, you can use the 'Similarity' propery of the examples.
    Low 'Similarity' values indicate the high similarity in strings.
    Return only the classification label starting with 'T' or 'T0000' if you found no appropriate class for 'InputText'.\n""" + xs
    
    #####
    
    #print(xs)
    prompt = xs
    result = llm.invoke(prompt)
    predicted_label = result.content
    return predicted_label


### Predicting single TTP label
def predict_label1(text_to_predict, train_vector_db, llm):
    ##### Prompt 1
    search_results = train_vector_db.similarity_search_with_score(text_to_predict, k=200)
    ttp_id_counts = {}
    
    # Initialize a document counter
    doc_counter = 0
    
    #xs = ["\nText:"+ doc.page_content.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results if score < 2]
    
    xs = [
        "\nText:" + doc.page_content.lower().strip() +
        "\nSimilarity:" + str(score) +
        "\nClassification:" + doc.metadata['TTP_ID'] + "\n"
        for doc, score in search_results
        if score < 2 and
        ttp_id_counts.setdefault(doc.metadata['TTP_ID'], 0) < 5 and
        not ttp_id_counts.update({doc.metadata['TTP_ID']: ttp_id_counts[doc.metadata['TTP_ID']] + 1})# and
        #not (doc_counter := doc_counter + 1)
    ]
    

    xs = ''.join(xs)+"\nInputText:"+ text_to_predict.lower().strip() +"\nClassification:"
    xs = """You are an expert in MITRE ATT&CK TTP classification.
    Your task is to classify the 'InputText' based on the provided examples below.
    
    Each example includes:
        - A sentence.
        - An inverse similarity score with the 'InputText' (lower scores indicate higher similarity).
        - A corresponding classification label starting with 'T'.
    
    Use these examples to determine the correct classification for the given text (InputText).
    
    If 'InputText' is completely not related to any of the provided examples, return class 'T0000'.
    To determine if the 'InputText' is not related to the given examples, you can use the 'Similarity' propery of the examples.
    Low 'Similarity' values indicate the high similarity in strings.
    Return only the classification label starting with 'T' or 'T0000' if you found no appropriate class for 'InputText'.\n""" + xs
    
    #####
    #**Think through the classification process step-by-step, comparing the 'InputText' to each example, but do not share your reasoning.**
    
    #print(xs)
    #print(doc_counter)
    prompt = xs
    result = llm.invoke(prompt)
    predicted_label = result.content
    return predicted_label



### Predicting single TTP label
def predict_label12(text_to_predict, train_vector_db, llm, distance=0.7):
    ##### Prompt 1
    search_results = train_vector_db.similarity_search_with_score(text_to_predict, k=200)
    ttp_id_counts = {}
    
    # Initialize a document counter
    doc_counter = 0
    
    #xs = ["\nText:"+ doc.page_content.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results if score < 2]
    
    xs = [
        "\nText:" + doc.page_content.lower().strip() +
        "\nSimilarity:" + str(score) +
        "\nClassification:" + doc.metadata['TTP_ID'] + "\n"
        for doc, score in search_results
        if score < distance and
        ttp_id_counts.setdefault(doc.metadata['TTP_ID'], 0) < 5 and
        not ttp_id_counts.update({doc.metadata['TTP_ID']: ttp_id_counts[doc.metadata['TTP_ID']] + 1})# and
        #not (doc_counter := doc_counter + 1)
    ]
    

    xs = ''.join(xs)+"\nInputText:"+ text_to_predict.lower().strip() +"\nClassification:"
    xs = """You are an expert in MITRE ATT&CK TTP classification.
    Your task is to classify the 'InputText' based on the provided examples below.
    
    Each example includes:
        - A sentence.
        - An inverse similarity score with the 'InputText' (lower scores indicate higher similarity).
        - A corresponding classification label starting with 'T'.
    
    Use these examples to determine the correct classification for the given text (InputText).
    If no samples are listed, return class 'T0000'.
    
    If 'InputText' is completely not related to any of the provided examples, return class 'T0000'.
    To determine if the 'InputText' is not related to the given examples, you can use the 'Similarity' propery of the examples.
    Low 'Similarity' values indicate the high similarity in strings.
    Return only the classification label starting with 'T' or 'T0000' if you found no appropriate class for 'InputText'.\n""" + xs
    
    #####
    #**Think through the classification process step-by-step, comparing the 'InputText' to each example, but do not share your reasoning.**
    
    #print(xs)
    #print(doc_counter)
    prompt = xs
    result = llm.invoke(prompt)
    predicted_label = result.content
    return predicted_label



### Enable predicting of Multiple TTP Labels
def predict_label2(text_to_predict, train_vector_db, llm):
    print(text_to_predict)
    ##### Prompt 1
    search_results = train_vector_db.similarity_search_with_score(text_to_predict, k=65)

    
    xs = ["\nText:"+ doc.page_content.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results if score < 10]
    #xs = ["\nText:'"+ doc.page_content.lower().strip()+"'\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results]
    

    xs = ''.join(xs)+"\nInputText:"+ text_to_predict.lower().strip() +"\nClassification:"
    xs = """You are an expert in MITRE ATT&CK TTP classification. 
    Your task is to classify the 'InputText' based on the examples provided below.

        Each example includes:
        
        - A sentence.
        - An inverse similarity score with the 'InputText' (lower scores indicate higher similarity).
        - A corresponding classification label starting with 'T'.
        
        Use these examples to determine the correct classification label(s) for the given 'InputText'.
        If 'InputText' is completely unrelated to any of the examples, return the classification label 'T0000'.
        
        To determine if 'InputText' is unrelated, refer to the 'Similarity' property of the examples.
        Lower similarity values indicate greater similarity between strings.
        
        Return only the classification label(s) starting with 'T'.
        If multiple classifications apply, list them in the format: T1111, T2222, ...".
        If no appropriate classification is found, return 'T0000'.
        Do not return anything but the classification labels!\n""" + xs
    
    #####
    
    print(xs)
    prompt = xs
    result = llm.invoke(prompt)
    predicted_label = result.content
    return predicted_label

### Zero-Shot Predictions
def predict_label_zero(text_to_predict, llm):
    ##### Prompt 1
    #search_results = train_vector_db.similarity_search_with_score(text_to_predict, k=65)

    
    #xs = ["\nText:"+ doc.page_content.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results if score < 2]
    #xs = ["\nText:'"+ doc.page_content.lower().strip()+"'\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results]
    

    
    xs = """You are an expert in MITRE ATT&CK TTP classification. 
    Your task is to classify the 'InputText'.

        If 'InputText' is completely unrelated to cybersecurity, return the classification label 'T0000'.
        
        Return only the classification label(s) starting with 'T'.
        If no appropriate classification is found, return 'T0000'.
        Do not return anything but the classification label!\n"""# + xs
    xs = ''.join(xs)+"\nInputText:"+ text_to_predict.lower().strip() +"\nClassification:"
    
    #####
    
    #print(xs)
    prompt = xs
    result = llm.invoke(prompt)
    predicted_label = result.content
    return predicted_label

In [110]:
from typing import Literal

def ttf_shot_predict(sentences, option: Literal["SINGLE_LABEL", "SINGLE_LABEL2", "COT", "MULTIPLE_LABEL", "ZERO_SHOT"], llm, db, distance=0.7):
    #print(sentences)
    predicted_labels = []
    # Iterating over the DataFrame row by row
    for sentence in sentences:
        
        match option:
            # single label
            case "SINGLE_LABEL":
                predicted_label = predict_label1(sentence, db, llm)
            case "SINGLE_LABEL2":
                predicted_label = predict_label12(sentence, db, llm, distance=distance)
            # multiple label
            case "MULTIPLE_LABEL":
                predicted_label = predict_label2(sentence, db, llm)
            # zero shot
            case "ZERO_SHOT":
                predicted_label = predict_label_zero(sentence, llm)
            case _:
                raise ValueError("Invalid option")
            
        predicted_labels.append(predicted_label)
        #predicted_labels.extend(predicted_label)

    cleaned_labels = [label.replace('Classification:', '') for label in predicted_labels]
    cleaned_labels_set = set(cleaned_labels)
    cleaned_labels_set.discard('T0000') #  remove empty classification
    #print(cleaned_labels_set)
    return cleaned_labels_set

In [96]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

def get_important_sentences(article):
    # Vectorize sentences
    sentences = sent_tokenize(article)#article.split('. ')
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)

    # Get sentence scores and select top sentences
    scores = X.sum(axis=1).flatten().tolist()[0]
    important_sentences = [sentences[i] for i in np.argsort(scores)[-5:][::-1]]

    #print("Important Sentences:", important_sentences)
    return important_sentences


def get_important_sentences2(article, n=5, score_threshold=0.1):
    """
    Extract important sentences from an article using TF-IDF scores.

    Parameters:
    - article (str): The input article text.
    - n (int): Number of top sentences to return based on TF-IDF scores.
    - score_threshold (float): Minimum score threshold for a sentence to be considered important.

    Returns:
    - important_sentences (list): List of important sentences.
    """
    # Tokenize sentences
    sentences = sent_tokenize(article)  # Use NLTK's sentence tokenizer

    # Vectorize sentences using TF-IDF
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)

    # Calculate sentence scores by summing TF-IDF values for each sentence
    scores = np.array(X.sum(axis=1)).flatten()  # Convert to a flat array

    # Combine sentences and scores for easier filtering
    sentence_scores = [(sentences[i], scores[i]) for i in range(len(sentences))]

    # Filter sentences by threshold and sort by score (descending)
    filtered_sentences = [s for s in sentence_scores if s[1] >= score_threshold]
    filtered_sentences = sorted(filtered_sentences, key=lambda x: x[1], reverse=True)

    # Select top `n` sentences
    important_sentences = [s[0] for s in filtered_sentences[:n]]

    return important_sentences



def get_important_sentences3(article, n=20):
    # Parse and summarize
    text_to_classify = article
    parser = PlaintextParser.from_string(text_to_classify, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summary = summarizer(parser.document, n)  # Top 5 sentences

    important_sentences = [str(sentence) for sentence in summary]
    #print("Important Sentences:", important_sentences)
    return important_sentences

In [16]:
# T1082, T1049
txt2 = """
Payload scans system and network for system information and network connection discovery
"""
### T1016 15x

In [17]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [73]:
#print("SINGLE_LABEL:", ttf_shot_predict(sent_tokenize(txt2), "SINGLE_LABEL", llm, db))

In [78]:
print("SINGLE_LABEL:", ttf_shot_predict(get_important_sentences(txt2), "SINGLE_LABEL", llm, db))

['\nPayload scans system and network for system information and network connection discovery']
SINGLE_LABEL: {'T1049'}


In [51]:
print("SINGLE_LABEL:", ttf_shot_predict([txt2], "MULTIPLE_LABEL", llm, db))

SINGLE_LABEL: {'T1082, T1049'}


In [103]:
print("SINGLE_LABEL:", ttf_shot_predict(get_important_sentences(txt2), "SINGLE_LABEL2", llm, db))

SINGLE_LABEL: {'T1049'}


In [104]:
print("SINGLE_LABEL:", ttf_shot_predict(["hi, how are you?"], "SINGLE_LABEL2", llm, db))

SINGLE_LABEL: set()


In [36]:
df_reports = pd.read_csv("../TTP-crawler/CISA-crawl-rt-ttp-ct.csv")
df_reports

,Unnamed: 0,RawText,TTP,CleanText
0,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\n\n\nActions to take today to mitigat...,"{'T1083', 'T1071.002', 'T1055.004', 'T1657', '...",\n\n\n\n\n\n\nActions to take today to mitigat...
1,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\nHow SVR-Attributed Actors are Adapting...,"{'T1621', 'T1090.002', 'T1110', 'T1078.004', '...",\n\n\n\nHow SVR-Attributed Actors are Adapting...
2,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\n\n\nActions to take today to mitigat...,"{'T1021.007', 'T1083', 'T1087.002', 'T1552.001...",\n\n\n\n\n\n\nActions to take today to mitigat...
3,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\n\n\nActions to take today to mitigat...,"{'TA0003', 'TA0008', 'T1083', 'T1654', 'T1078....",\n\n\n\n\n\n\nActions to take today to mitigat...
4,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\n\n\nActions to take today to mitigat...,"{'TA0006', 'T1595.002', 'T1190', 'T1083', 'T15...",\n\n\n\n\n\n\nActions to take today to mitigat...
...,...,...,...,...
72,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\nSummary\n\nThis joint cybersecurity ...,"{'TA0003', 'T1190', 'TA0008', 'T1110', 'T1189'...",\n\n\n\n\nSummary\n\nThis joint cybersecurity ...
73,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\nSummary\n\nThis Alert uses the MITRE...,"{'T1041', 'T1573.002', 'T1566.002', 'T1021.002...",\n\n\n\n\nSummary\n\nThis Alert uses the MITRE...
74,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\nSummary\n\nThis Alert uses the MITRE...,"{'T1041', 'T1055.012', 'T1027.002', 'T1546.008...",\n\n\n\n\nSummary\n\nThis Alert uses the MITRE...
75,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\nSummary\n\nThis Alert uses the MITRE...,"{'T1083', 'T1016', 'T1036.005', 'T1105', 'T108...",\n\n\n\n\nSummary\n\nThis Alert uses the MITRE...


In [41]:
df_reports = df_reports.head(2)

In [105]:
df_reports

,Unnamed: 0,RawText,TTP,CleanText,ttpfshot_sl,ttpfshot_ml2,ttpfshot_sli,ttpfshot_sli2
0,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\n\n\nActions to take today to mitigat...,"{'T1083', 'T1071.002', 'T1055.004', 'T1657', '...",\n\n\n\n\n\n\nActions to take today to mitigat...,"{T1567.002, T1102.002, T1219, T1550.002, T1548...",{T1490},"{T1567.002, T1548.003, T1219, T1550.002, T1105...","{T1567.002, T1548.003, T1219, T1105, T1076, T1..."
1,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\nHow SVR-Attributed Actors are Adapting...,"{'T1621', 'T1090.002', 'T1110', 'T1078.004', '...",\n\n\n\nHow SVR-Attributed Actors are Adapting...,"{T1090.001, T1591.004, T1070, T1586.003, T1136...",{T1490},,


In [42]:
## DSC Benchmarking - TTPXHunter vs. TTPFShot

In [106]:
import time

#df_reports["ttpxhunter"] = ""
df_reports["ttpfshot_sl"] = ""
#df_reports["ttpfshot_ml"] = ""
#df_reports["ttpfshot_zs"] = ""

for index, row in df_reports.iterrows():
    #print(f"Index: {index}")
    print("text nr: ", index)
    
    text_to_classify = row["CleanText"]
 
    
    start_time = time.time()
    ttpfshot_sl = ttf_shot_predict(sent_tokenize(text_to_classify), "SINGLE_LABEL2", llm, db)
    elapsed_time = (time.time() - start_time)/60
    print(f"ttpfshot_sl done in: {elapsed_time} minutes")
    
    '''
    start_time = time.time()
    ttpfshot_ml = ttf_shot_predict(sent_tokenize(text_to_classify), "MULTIPLE_LABEL", llm, db3)
    elapsed_time = (time.time() - start_time)/60
    print(f"ttpfshot_ml done in: {elapsed_time} minutes")
    
    start_time = time.time()
    ttpfshot_zs = ttf_shot_predict(sent_tokenize(text_to_classify), "ZERO_SHOT", llm, db3)
    elapsed_time = (time.time() - start_time)/60
    print(f"ttpfshot_zs done in: {elapsed_time} minutes")
    
    start_time = time.time()
    ttps, ttp_names = extract_ttp_from_sentences(sent_tokenize(text_to_classify), th, label_dict, ttpid2name)#(text_to_classify, th, label_dict, ttpid2name)
    elapsed_time = (time.time() - start_time)/60
    print(f"ttpxhunter done in: {elapsed_time} minutes")
    '''
    ### save results to the reports dataframe
    #df_reports.at[index, 'ttpxhunter'] = ttps
    df_reports.at[index, 'ttpfshot_sl'] = ttpfshot_sl
    #df_reports.at[index, 'ttpfshot_ml'] = ttpfshot_ml
    #df_reports.at[index, 'ttpfshot_zs'] = ttpfshot_zs
   
    
    #print("ttpxhunter:", ttps)
    #print("ttpxhunter:", ttps)
    print("ttpfshot_sl:", ttpfshot_sl)
    #print("ttpfshot_ml:", ttpfshot_ml)
    #print("ttpZshot_zs:", ttpfshot_zs)
    
    #TTPFShot
    

text nr:  0
ttpfshot_sl done in: 3.0470416227976482 minutes
ttpfshot_sl: {'T1567.002', 'T1102.002', 'T1219', 'T1550.002', 'T1548.003', 'T1105', 'T1489', 'T1076', 'T1083', 'T1070.004', 'T1027', 'T1110.003', 'T1562.011', 'T1140', 'T1562.004', 'T1201', 'T1562.001', 'T1110.001', 'T1598.004', 'T1552.001', 'T1490', 'T1553', 'T1537', 'T1021.001', 'T1070.001', 'T1588.001', 'T1560', 'T1599', 'T1486', 'T1053.005', 'T1094', 'T1059.003', 'T1553.006', 'T1059', 'T1016', 'T1555.003', 'T1556.002', 'T1586', 'T1657', 'T1547.001', 'T1003.005', 'T1087', 'T1556.004', 'T1562.009', 'T1591', 'T1055.001', 'T1048.003', 'T1069.002', 'T1218.005', 'T1136.001', 'T1597.001', 'T1021', 'T1555.005'}
text nr:  1
ttpfshot_sl done in: 2.0276062448819476 minutes
ttpfshot_sl: {'T1090.001', 'T1591.004', 'T1548.003', 'T1070', 'T1136.003', 'T1586.003', 'T1098.005', 'T1110.003', 'T1621', 'T1550.001', 'T1201', 'T1588.002', 'T1071.001', 'T1078.003', 'T1665', 'T1190', 'T1528', 'T1599', 'T1108', 'T1589.002', 'T1029', 'T1111', 'T107

In [44]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, hamming_loss, f1_score, precision_score, recall_score, jaccard_score

def calculate_metrics2(df, y_true_col, y_pred_col, key):
    # Convert columns to lists of sets
    y_true = df[y_true_col].apply(lambda x: eval(x) if isinstance(x, str) else x).tolist()
    y_pred = df[y_pred_col].tolist()

    # Define all unique TTPs across the entire dataset
    all_ttps = set.union(*y_true, *y_pred)

    # Use MultiLabelBinarizer to transform labels
    mlb = MultiLabelBinarizer(classes=list(all_ttps))
    y_true_bin = mlb.fit_transform(y_true)
    y_pred_bin = mlb.transform(y_pred)

    # Calculate metrics
    metrics = {
        "subset_accuracy": accuracy_score(y_true_bin, y_pred_bin),
        "hamming_accuracy": 1 - hamming_loss(y_true_bin, y_pred_bin),
        "f1_micro": f1_score(y_true_bin, y_pred_bin, average='micro'),
        "f1_macro": f1_score(y_true_bin, y_pred_bin, average='macro'),
        "f1_weighted": f1_score(y_true_bin, y_pred_bin, average='weighted'),
        "precision_micro": precision_score(y_true_bin, y_pred_bin, average='micro'),
        "precision_macro": precision_score(y_true_bin, y_pred_bin, average='macro'),
        "recall_micro": recall_score(y_true_bin, y_pred_bin, average='micro'),
        "recall_macro": recall_score(y_true_bin, y_pred_bin, average='macro'),
        "jaccard_micro": jaccard_score(y_true_bin, y_pred_bin, average='micro'),
        "jaccard_macro": jaccard_score(y_true_bin, y_pred_bin, average='macro')
    }
    
    # Return the metrics dictionary for the specified key
    return {key: metrics}


def calculate_single_label_metrics2(df, y_true_col, y_pred_col, key):
    # Convert columns to lists (no need for sets or multi-label encoding)
    y_true = df[y_true_col].tolist()
    y_pred = df[y_pred_col].tolist()

    # Calculate metrics for single-label classification
    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "hamming_accuracy": 1 - hamming_loss(y_true, y_pred),
        "f1_micro": f1_score(y_true, y_pred, average='micro'),
        "f1_macro": f1_score(y_true, y_pred, average='macro'),
        "f1_weighted": f1_score(y_true, y_pred, average='weighted'),
        "precision_micro": precision_score(y_true, y_pred, average='micro'),
        "precision_macro": precision_score(y_true, y_pred, average='macro'),
        "recall_micro": recall_score(y_true, y_pred, average='micro'),
        "recall_macro": recall_score(y_true, y_pred, average='macro'),
        "jaccard_micro": jaccard_score(y_true, y_pred, average='micro'),
        "jaccard_macro": jaccard_score(y_true, y_pred, average='macro')
    }
    
    # Return the metrics dictionary for the specified key
    return {key: metrics}

In [45]:

all_scores = {
    #"ttpxhunter" : calculate_metrics2(df_reports, 'TTP', 'ttpxhunter', 'ttpxhunter'),
    "ttpfshot_sl" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_sl', 'ttpfshot_sl'),
    #"ttpfshot_ml" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_ml', 'ttpfshot_ml'),
    #"ttpfshot_zs" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_zs', 'ttpfshot_zs')
}

#scores_df = pd.DataFrame(all_scores)#.transpose()

#print(scores_df)

flattened_data = {outer_key: inner_dict[outer_key] for outer_key, inner_dict in all_scores.items()}

# Create DataFrame
scores_df = pd.DataFrame(flattened_data).transpose()
scores_df

/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,subset_accuracy,hamming_accuracy,f1_micro,f1_macro,f1_weighted,precision_micro,precision_macro,recall_micro,recall_macro,jaccard_micro,jaccard_macro
ttpfshot_sl,0.0,0.561404,0.305556,0.187135,0.418301,0.236559,0.184211,0.431373,0.192982,0.180328,0.184211


### SINGLE_LABEL 2 - trehsold 0.7

In [107]:

all_scores = {
    #"ttpxhunter" : calculate_metrics2(df_reports, 'TTP', 'ttpxhunter', 'ttpxhunter'),
    "ttpfshot_sl" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_sl', 'ttpfshot_sl'),
    #"ttpfshot_ml" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_ml', 'ttpfshot_ml'),
    #"ttpfshot_zs" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_zs', 'ttpfshot_zs')
}

#scores_df = pd.DataFrame(all_scores)#.transpose()

#print(scores_df)

flattened_data = {outer_key: inner_dict[outer_key] for outer_key, inner_dict in all_scores.items()}

# Create DataFrame
scores_df = pd.DataFrame(flattened_data).transpose()
scores_df

/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,subset_accuracy,hamming_accuracy,f1_micro,f1_macro,f1_weighted,precision_micro,precision_macro,recall_micro,recall_macro,jaccard_micro,jaccard_macro
ttpfshot_sl,0.0,0.567308,0.318182,0.201923,0.411765,0.259259,0.201923,0.411765,0.201923,0.189189,0.201923


### Important sentence Prediction

In [91]:
import time

#df_reports["ttpxhunter"] = ""
df_reports["ttpfshot_sli"] = ""
#df_reports["ttpfshot_ml2"] = ""
#df_reports["ttpfshot_ml"] = ""
#df_reports["ttpfshot_zs"] = ""

for index, row in df_reports.iterrows():
    #print(f"Index: {index}")
    print("text nr: ", index)
    
    text_to_classify = row["CleanText"]
 
   
    start_time = time.time()
    ttpfshot_sli = ttf_shot_predict(get_important_sentences2(text_to_classify, n=60), "SINGLE_LABEL", llm, db)
    elapsed_time = (time.time() - start_time)/60
    print(f"ttpfshot_sl done in: {elapsed_time} minutes")
    '''
    
    start_time = time.time()
    ttpfshot_ml2 = ttf_shot_predict([text_to_classify], "MULTIPLE_LABEL", llm, db)
    elapsed_time = (time.time() - start_time)/60
    print(f"ttpfshot_ml done in: {elapsed_time} minutes")
    
    start_time = time.time()
    ttpfshot_zs = ttf_shot_predict(sent_tokenize(text_to_classify), "ZERO_SHOT", llm, db3)
    elapsed_time = (time.time() - start_time)/60
    print(f"ttpfshot_zs done in: {elapsed_time} minutes")
    
    start_time = time.time()
    ttps, ttp_names = extract_ttp_from_sentences(sent_tokenize(text_to_classify), th, label_dict, ttpid2name)#(text_to_classify, th, label_dict, ttpid2name)
    elapsed_time = (time.time() - start_time)/60
    print(f"ttpxhunter done in: {elapsed_time} minutes")
    '''
    ### save results to the reports dataframe
    #df_reports.at[index, 'ttpxhunter'] = ttps
    #df_reports.at[index, 'ttpfshot_sl'] = ttpfshot_sl
    df_reports.at[index, 'ttpfshot_sli'] = ttpfshot_sli
    #df_reports.at[index, 'ttpfshot_zs'] = ttpfshot_zs
   
    
    #print("ttpxhunter:", ttps)
    #print("ttpxhunter:", ttps)
    #print("ttpfshot_sl:", ttpfshot_sl)
    print("ttpfshot_sli:", ttpfshot_sli)
    #print("ttpZshot_zs:", ttpfshot_zs)
    
    #TTPFShot

text nr:  0
ttpfshot_sl done in: 1.4069068074226379 minutes
ttpfshot_sli: {'T1548.003', 'T1219', 'T1550.002', 'T1105', 'T1083', 'T1076', 'T1021.002', 'T1027', 'T1140', 'T1562.004', 'T1201', 'T1562.001', 'T1110.001', 'T1518.001', 'T1490', 'T1588.001', 'T1486', 'T1053.005', 'T1094', 'T1059.003', 'T1059', 'T1657', 'T1547.001', 'T1003.005', 'T1556.004', 'T1562.009', 'T1591', 'T1055.001', 'T1048.003', 'T1069.002', 'T1597.001'}
text nr:  1
ttpfshot_sl done in: 1.3267831524213156 minutes
ttpfshot_sli: {'T1591.004', 'T1070', 'T1548.003', 'T1586.003', 'T1136.003', 'T1583.003', 'T1098.005', 'T1580', 'T1110.003', 'T1594', 'T1556.006', 'T1597.001', 'T1621', 'T1550', 'T1588.002', 'T1071.001', 'T1078.003', 'T1665', 'T1190', 'T1528', 'T1599', 'T1649', 'T1108', 'T1589.002', 'T1583', 'T1527', 'T1029', 'T1111', 'T1078', 'T1090.002', 'T1036', 'T1098.001', 'T1195.002', 'T1078.004'}


In [92]:

all_scores = {
    #"ttpxhunter" : calculate_metrics2(df_reports, 'TTP', 'ttpxhunter', 'ttpxhunter'),
    "ttpfshot_sli" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_sli', 'ttpfshot_sli'),
    #"ttpfshot_ml" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_ml', 'ttpfshot_ml'),
    #"ttpfshot_zs" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_zs', 'ttpfshot_zs')
}

#scores_df = pd.DataFrame(all_scores)#.transpose()

#print(scores_df)

flattened_data = {outer_key: inner_dict[outer_key] for outer_key, inner_dict in all_scores.items()}

# Create DataFrame
scores_df = pd.DataFrame(flattened_data).transpose()
scores_df

/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,subset_accuracy,hamming_accuracy,f1_micro,f1_macro,f1_weighted,precision_micro,precision_macro,recall_micro,recall_macro,jaccard_micro,jaccard_macro
ttpfshot_sli,0.0,0.557895,0.275862,0.168421,0.313725,0.246154,0.168421,0.313725,0.168421,0.16,0.168421


In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


for index, row in df_reports.iterrows():
    #print(f"Index: {index}")
    print("text nr: ", index)
    
    text_to_classify = row["CleanText"]
    # Vectorize sentences
    sentences = sent_tokenize(text_to_classify)#article.split('. ')
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)

    # Get sentence scores and select top sentences
    scores = X.sum(axis=1).flatten().tolist()[0]
    important_sentences = [sentences[i] for i in np.argsort(scores)[-5:][::-1]]

    print("Important Sentences:", important_sentences)

text nr:  0
Important Sentences: ['CISA: Known Exploited Vulnerabilities Catalog\nCISA, MITRE: Best Practices for MITRE ATT&CK Mapping\nCISA: Decider Tool\nCISA: Cross-Sector Cybersecurity Performance Goals\nCISA: Secure by Design\nCISA: Implementing Phishing-Resistant MFA\nCISA: Guide to Securing Remote Access Software\n\nREFERENCES,1, Privacy Affairs: “Moral” 8Base Ransomware Targets 2 New Victims,2, VMware: 8base ransomware: A Heavy Hitting Player,3, Infosecurity Magazine: Phobos Ransomware Family Expands With New FAUST Variant,4, The Record: Hospitals offline across Romania following ransomware attack on IT platform,5, Comparitech: What is Phobos Ransomware & How to Protect Against It?,6, Cisco Talos: Understanding the Phobos affiliate structure and activity,7, Cisco Talos: A deep dive into Phobos ransomware, recently deployed by 8Base group,8, Malwarebytes Labs: A deep dive into Phobos ransomware,9, Any Run: Smokeloader,10, Malpedia: Smokeloader,11, Truesec:\xa0A case of the FAUST

In [82]:
%pip install sumy --break-system-packages

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 2.8 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=0ef6fa5bf3eda91ad511583c9f2c9d70cd9ba2fd84718d66e6d83d38b9c91b06
  Stored in directory: /home/hamzicd/.cache/pip/wheels/32/99/64/59305409cacd03aa03e7bddf31a9db34b1fa7033bd41972662
Successfully built breadability
Note: you may need to restart the kernel to use updated packages.


In [84]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

# Parse and summarize

for index, row in df_reports.iterrows():
    #print(f"Index: {index}")
    print("text nr: ", index)
    
    text_to_classify = row["CleanText"]
    parser = PlaintextParser.from_string(text_to_classify, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summary = summarizer(parser.document, 20)  # Top 5 sentences

    print("Important Sentences:", [str(sentence) for sentence in summary])

text nr:  0
Important Sentences: ['The Federal Bureau of Investigation (FBI), the Cybersecurity and Infrastructure Security Agency (CISA), and the Multi-State Information Sharing and Analysis Center (MS-ISAC) are releasing this joint CSA, to disseminate known TTPs and IOCs associated with the Phobos ransomware variants observed as recently as February 2024, according to open source reporting.', 'These incidents targeted municipal and county governments, emergency services, education, public healthcare, and other critical infrastructure entities to successfully ransom several million U.S. dollars.,1,2, The FBI, CISA, and the MS-ISAC encourage organizations to implement the recommendations in the Mitigations section of this CSA to reduce the likelihood and impact of Phobos ransomware and other ransomware incidents.', 'See the MITRE ATT&CK Tactics and Techniques section for a table of the threat actors’ activity mapped to MITRE ATT&CK® tactics and techniques.', 'For assistance with mappin

### Varying Article's number of important sentences

In [97]:
df_reports

,Unnamed: 0,RawText,TTP,CleanText,ttpfshot_sl,ttpfshot_ml2,ttpfshot_sli
0,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\n\n\nActions to take today to mitigat...,"{'T1083', 'T1071.002', 'T1055.004', 'T1657', '...",\n\n\n\n\n\n\nActions to take today to mitigat...,"{T1567.002, T1102.002, T1219, T1550.002, T1548...",{T1490},"{T1548.003, T1219, T1550.002, T1105, T1083, T1..."
1,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\nHow SVR-Attributed Actors are Adapting...,"{'T1621', 'T1090.002', 'T1110', 'T1078.004', '...",\n\n\n\nHow SVR-Attributed Actors are Adapting...,"{T1090.001, T1591.004, T1070, T1586.003, T1136...",{T1490},"{T1591.004, T1070, T1548.003, T1586.003, T1136..."


In [99]:
import time


for n_top_sent in [10,20,30,40,50,60,80,100]:

    #df_reports["ttpxhunter"] = ""
    df_reports["ttpfshot_sli"] = ""
    df_reports["ttpfshot_sli2"] = ""
    #df_reports["ttpfshot_ml2"] = ""
    #df_reports["ttpfshot_ml"] = ""
    #df_reports["ttpfshot_zs"] = ""

    for index, row in df_reports.iterrows():
        #print(f"Index: {index}")
        print("text nr: ", index)

        text_to_classify = row["CleanText"]


        start_time = time.time()
        ttpfshot_sli = ttf_shot_predict(get_important_sentences2(text_to_classify, n=n_top_sent), "SINGLE_LABEL", llm, db)
        elapsed_time = (time.time() - start_time)/60
        print(f"ttpfshot_sl done in: {elapsed_time} minutes")
        
        start_time = time.time()
        ttpfshot_sli2 = ttf_shot_predict(get_important_sentences3(text_to_classify, n=n_top_sent), "SINGLE_LABEL", llm, db)
        elapsed_time = (time.time() - start_time)/60
        print(f"ttpfshot_sl done in: {elapsed_time} minutes")
        '''

        start_time = time.time()
        ttpfshot_ml2 = ttf_shot_predict([text_to_classify], "MULTIPLE_LABEL", llm, db)
        elapsed_time = (time.time() - start_time)/60
        print(f"ttpfshot_ml done in: {elapsed_time} minutes")

        start_time = time.time()
        ttpfshot_zs = ttf_shot_predict(sent_tokenize(text_to_classify), "ZERO_SHOT", llm, db3)
        elapsed_time = (time.time() - start_time)/60
        print(f"ttpfshot_zs done in: {elapsed_time} minutes")

        start_time = time.time()
        ttps, ttp_names = extract_ttp_from_sentences(sent_tokenize(text_to_classify), th, label_dict, ttpid2name)#(text_to_classify, th, label_dict, ttpid2name)
        elapsed_time = (time.time() - start_time)/60
        print(f"ttpxhunter done in: {elapsed_time} minutes")
        '''
        ### save results to the reports dataframe
        #df_reports.at[index, 'ttpxhunter'] = ttps
        #df_reports.at[index, 'ttpfshot_sl'] = ttpfshot_sl
        df_reports.at[index, 'ttpfshot_sli'] = ttpfshot_sli
        df_reports.at[index, 'ttpfshot_sli2'] = ttpfshot_sli2
        #df_reports.at[index, 'ttpfshot_zs'] = ttpfshot_zs


        #print("ttpxhunter:", ttps)
        #print("ttpxhunter:", ttps)
        #print("ttpfshot_sl:", ttpfshot_sl)
        print("ttpfshot_sli:", ttpfshot_sli)
        print("ttpfshot_sli2:", ttpfshot_sli2)
        #print("ttpZshot_zs:", ttpfshot_zs)

        #Evaluate
        all_scores = {
            #"ttpxhunter" : calculate_metrics2(df_reports, 'TTP', 'ttpxhunter', 'ttpxhunter'),
            "ttpfshot_sli" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_sli', 'ttpfshot_sli'),
            "ttpfshot_sli2" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_sli2', 'ttpfshot_sli2'),
            #"ttpfshot_ml" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_ml', 'ttpfshot_ml'),
            #"ttpfshot_zs" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_zs', 'ttpfshot_zs')
        }

        #scores_df = pd.DataFrame(all_scores)#.transpose()

        #print(scores_df)

        flattened_data = {outer_key: inner_dict[outer_key] for outer_key, inner_dict in all_scores.items()}

        # Create DataFrame
        scores_df = pd.DataFrame(flattened_data).transpose()
        print(scores_df)

text nr:  0
ttpfshot_sl done in: 0.2899808605511983 minutes
ttpfshot_sl done in: 0.43698720932006835 minutes
ttpfshot_sli: {'T1140', 'T1219', 'T1547.001', 'T1486', 'T1094', 'T1055.001', 'T1110.001'}
ttpfshot_sli2: {'T1486', 'T1094', 'T1055.001', 'T1110.001'}


/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/py

               subset_accuracy  hamming_accuracy  f1_micro  f1_macro  \
ttpfshot_sli               0.0          0.528302  0.137931  0.075472   
ttpfshot_sli2              0.0          0.500000  0.036364  0.018868   

               f1_weighted  precision_micro  precision_macro  recall_micro  \
ttpfshot_sli      0.078431         0.571429         0.075472      0.078431   
ttpfshot_sli2     0.019608         0.250000         0.018868      0.019608   

               recall_macro  jaccard_micro  jaccard_macro  
ttpfshot_sli       0.075472       0.074074       0.075472  
ttpfshot_sli2      0.018868       0.018519       0.018868  
text nr:  1
ttpfshot_sl done in: 0.3100603381792704 minutes
ttpfshot_sl done in: 0.21674156586329144 minutes
ttpfshot_sli: {'T1556.006', 'T1190', 'T1599', 'T1071.001', 'T1589.002', 'T1078.004'}
ttpfshot_sli2: {'T1621', 'T1599', 'T1548.005', 'T1583.003', 'T1098.005', 'T1580', 'T1078.004'}
               subset_accuracy  hamming_accuracy  f1_micro  f1_macro  \
ttpfsho

/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/py

ttpfshot_sl done in: 0.5507293621699015 minutes
ttpfshot_sl done in: 0.5495056867599487 minutes
ttpfshot_sli: {'T1140', 'T1219', 'T1588.001', 'T1547.001', 'T1486', 'T1094', 'T1591', 'T1055.001', 'T1110.001'}
ttpfshot_sli2: {'T1005', 'T1547.001', 'T1105', 'T1486', 'T1094', 'T1591', 'T1055.001', 'T1110.001', 'T1490'}
               subset_accuracy  hamming_accuracy  f1_micro  f1_macro  \
ttpfshot_sli               0.0          0.527273  0.133333  0.072727   
ttpfshot_sli2              0.0          0.527273  0.133333  0.072727   

               f1_weighted  precision_micro  precision_macro  recall_micro  \
ttpfshot_sli      0.078431         0.444444         0.072727      0.078431   
ttpfshot_sli2     0.078431         0.444444         0.072727      0.078431   

               recall_macro  jaccard_micro  jaccard_macro  
ttpfshot_sli       0.072727       0.071429       0.072727  
ttpfshot_sli2      0.072727       0.071429       0.072727  
text nr:  1


...

ttpfshot_sl done in: 1.2041556398073832 minutes
ttpfshot_sl done in: 1.8261729637781778 minutes
ttpfshot_sli: {'T1591.004', 'T1070', 'T1548.003', 'T1586.003', 'T1136.003', 'T1583.003', 'T1098.005', 'T1580', 'T1110.003', 'T1556.006', 'T1597.001', 'T1621', 'T1550', 'T1550.001', 'T1588.002', 'T1071.001', 'T1078.003', 'T1591.002', 'T1665', 'T1190', 'T1528', 'T1599', 'T1649', 'T1108', 'T1589.002', 'T1583', 'T1029', 'T1111', 'T1078', 'T1090.002', 'T1036', 'T1098.001', 'T1195.002', 'T1078.004'}
ttpfshot_sli2: {'T1090.001', 'T1591.004', 'T1548.003', 'T1070', 'T1136.003', 'T1586.003', 'T1583.003', 'T1098.005', 'T1580', 'T1110.003', 'T1526', 'T1621', 'T1550', 'T1588.002', 'T1071.001', 'T1078.003', 'T1591.002', 'T1665', 'T1190', 'T1528', 'T1588.001', 'T1599', 'T1649', 'T1108', 'T1566.002', 'T1589.002', 'T1078', 'T1036', 'T1098.001', 'T1078.004'}
               subset_accuracy  hamming_accuracy  f1_micro  f1_macro  \
ttpfshot_sli               0.0          0.552632  0.273504  0.168421   
ttpfshot_

/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/py

ttpfshot_sl done in: 2.0096505165100096 minutes
ttpfshot_sl done in: 3.3639552275339764 minutes
ttpfshot_sli: {'T1567.002', 'T1548.003', 'T1219', 'T1550.002', 'T1105', 'T1083', 'T1076', 'T1021.002', 'T1027', 'T1562.011', 'T1140', 'T1562.004', 'T1218', 'T1201', 'T1555.004', 'T1562.001', 'T1110.001', 'T1591.001', 'T1518.001', 'T1598.004', 'T1490', 'T1588.001', 'T1486', 'T1053.005', 'T1094', 'T1059.003', 'T1059', 'T1016', 'T1555.003', 'T1556.002', 'T1657', 'T1547.001', 'T1111', 'T1562.009', 'T1591', 'T1055.001', 'T1048.003', 'T1069.002', 'T1485', 'T1597.001'}
ttpfshot_sli2: {'T1567.002', 'T1548.003', 'T1219', 'T1105', 'T1076', 'T1021.002', 'T1027', 'T1140', 'T1562.004', 'T1218', 'T1555.004', 'T1562.001', 'T1110.001', 'T1518.001', 'T1490', 'T1021.001', 'T1082', 'T1599', 'T1486', 'T1053.005', 'T1094', 'T1553.006', 'T1059', 'T1556.002', 'T1586', 'T1657', 'T1547.001', 'T1585.002', 'T1111', 'T1078', 'T1562.009', 'T1591', 'T1055.001', 'T1048.003', 'T1069.002', 'T1218.005', 'T1485', 'T1597.001'}

/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/py

KeyboardInterrupt: 

### Simulate different distance tresholds

In [113]:
for min_distance in [0.2,0.3,0.4,0.5,0.7,0.8]:
    print("min_distance:", min_distance)


    #df_reports["ttpxhunter"] = ""
    df_reports["ttpfshot_sl"] = ""
    #df_reports["ttpfshot_ml"] = ""
    #df_reports["ttpfshot_zs"] = ""

    for index, row in df_reports.iterrows():
        #print(f"Index: {index}")
        print("text nr: ", index)

        text_to_classify = row["CleanText"]


        start_time = time.time()
        ttpfshot_sl = ttf_shot_predict(sent_tokenize(text_to_classify), "SINGLE_LABEL2", llm, db, distance=min_distance)
        elapsed_time = (time.time() - start_time)/60
        print(f"ttpfshot_sl done in: {elapsed_time} minutes")

        '''
        start_time = time.time()
        ttpfshot_ml = ttf_shot_predict(sent_tokenize(text_to_classify), "MULTIPLE_LABEL", llm, db3)
        elapsed_time = (time.time() - start_time)/60
        print(f"ttpfshot_ml done in: {elapsed_time} minutes")

        start_time = time.time()
        ttpfshot_zs = ttf_shot_predict(sent_tokenize(text_to_classify), "ZERO_SHOT", llm, db3)
        elapsed_time = (time.time() - start_time)/60
        print(f"ttpfshot_zs done in: {elapsed_time} minutes")

        start_time = time.time()
        ttps, ttp_names = extract_ttp_from_sentences(sent_tokenize(text_to_classify), th, label_dict, ttpid2name)#(text_to_classify, th, label_dict, ttpid2name)
        elapsed_time = (time.time() - start_time)/60
        print(f"ttpxhunter done in: {elapsed_time} minutes")
        '''
        ### save results to the reports dataframe
        #df_reports.at[index, 'ttpxhunter'] = ttps
        df_reports.at[index, 'ttpfshot_sl'] = ttpfshot_sl
        #df_reports.at[index, 'ttpfshot_ml'] = ttpfshot_ml
        #df_reports.at[index, 'ttpfshot_zs'] = ttpfshot_zs


        #print("ttpxhunter:", ttps)
        #print("ttpxhunter:", ttps)
        print("ttpfshot_sl:", ttpfshot_sl)
        #print("ttpfshot_ml:", ttpfshot_ml)
        #print("ttpZshot_zs:", ttpfshot_zs)
        
    all_scores = {
        #"ttpxhunter" : calculate_metrics2(df_reports, 'TTP', 'ttpxhunter', 'ttpxhunter'),
        "ttpfshot_sl" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_sl', 'ttpfshot_sl'),
        #"ttpfshot_ml" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_ml', 'ttpfshot_ml'),
        #"ttpfshot_zs" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_zs', 'ttpfshot_zs')
    }

    #scores_df = pd.DataFrame(all_scores)#.transpose()

    #print(scores_df)

    flattened_data = {outer_key: inner_dict[outer_key] for outer_key, inner_dict in all_scores.items()}

    # Create DataFrame
    scores_df = pd.DataFrame(flattened_data).transpose()
    print(scores_df)
        

min_distance: 0.2
text nr:  0
ttpfshot_sl done in: 2.610564192136129 minutes
ttpfshot_sl: {'T1488', 'T1213', 'T1552', 'T1219', 'T1210', 'T1070', 'T1102', 'T1589', 'T1027', 'T1543', 'T1562', 'T1542', 'T1550', 'T1203', 'T1201', 'T1110', 'T1212', 'T1049', 'T1587', 'T1490', 'T1553', 'T1547', 'T1567', 'T1190', 'T1560', 'T1486', 'T1595', 'T1583', 'T1059', 'T1003', 'T1204', 'T1566', 'T1569', 'T1078', 'T1087', 'T1485', 'T1040', 'T1592', 'T1021'}
text nr:  1
ttpfshot_sl done in: 1.2081881364186604 minutes
ttpfshot_sl: {'T1589', 'T1531', 'T1090', 'T1562', 'T1071', 'T1200', 'T1550', 'T1556', 'T1110', 'T1193', 'T1190', 'T1583', 'T1195', 'T1529', 'T1111', 'T1233', 'T1078', 'T1087', 'T1036'}
             subset_accuracy  hamming_accuracy  f1_micro  f1_macro  \
ttpfshot_sl              0.0               0.5  0.146789    0.0681   

             f1_weighted  precision_micro  precision_macro  recall_micro  \
ttpfshot_sl     0.143791         0.137931         0.064516      0.156863   

             recall

/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ttpfshot_sl done in: 2.3340789755185445 minutes
ttpfshot_sl: {'T1567.002', 'T1213', 'T1219', 'T1210', 'T1105', 'T1102', 'T1489', 'T1083', 'T1076', 'T1589', 'T1070.004', 'T1027', 'T1110.003', 'T1562', 'T1071', 'T1140', 'T1201', 'T1110', 'T1212', 'T1562.001', 'T1049', 'T1587', 'T1490', 'T1480.001', 'T1553', 'T1190', 'T1588.001', 'T1560', 'T1486', 'T1053.005', 'T1595', 'T1094', 'T1135', 'T1059.003', 'T1003.001', 'T1657', 'T1547.001', 'T1566', 'T1569', 'T1078', 'T1087', 'T1562.009', 'T1485', 'T1055.001', 'T1048.003', 'T1069.002', 'T1592', 'T1218.005', 'T1136.001', 'T1040', 'T1021'}
text nr:  1
ttpfshot_sl done in: 1.320796791712443 minutes
ttpfshot_sl: {'T1070', 'T1098.005', 'T1531', 'T1110.003', 'T1584.008', 'T1090', 'T1071', 'T1200', 'T1621', 'T1550', 'T1556', 'T1110', 'T1078.003', 'T1587', 'T1193', 'T1665', 'T1528', 'T1190', 'T1589.002', 'T1195', 'T1029', 'T1555.003', 'T1078', 'T1087', 'T1036', 'T1578.005', 'T1133'}
             subset_accuracy  hamming_accuracy  f1_micro  f1_macro  \
t

/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ttpfshot_sl done in: 2.4592469930648804 minutes
ttpfshot_sl: {'T1567.002', 'T1548.003', 'T1219', 'T1550.002', 'T1105', 'T1489', 'T1076', 'T1083', 'T1021.002', 'T1070.004', 'T1027', 'T1555', 'T1562.004', 'T1201', 'T1588.002', 'T1110', 'T1555.004', 'T1588', 'T1562.001', 'T1110.001', 'T1548.002', 'T1518.001', 'T1587', 'T1598.004', 'T1490', 'T1070.001', 'T1560', 'T1599', 'T1486', 'T1595', 'T1094', 'T1059.003', 'T1566.002', 'T1553.006', 'T1059', 'T1016', 'T1555.003', 'T1657', 'T1547.001', 'T1216', 'T1071.003', 'T1562.009', 'T1591', 'T1218.005', 'T1048.003', 'T1069.002', 'T1556.', 'T1021', 'T1555.005'}
text nr:  1
ttpfshot_sl done in: 1.307725767294566 minutes
ttpfshot_sl: {'T1591.004', 'T1210', 'T1583.003', 'T1098.005', 'T1556.009', 'T1110.003', 'T1090', 'T1594', 'T1562', 'T1200', 'T1621', 'T1588.002', 'T1071.001', 'T1078.003', 'T1193', 'T1665', 'T1190', 'T1528', 'T1108', 'T1589.002', 'T1029', 'T1189', 'T1111', 'T1078', 'T1090.002', 'T1036', 'T1098.001', 'T1195.002', 'T1535', 'T1078.004'}
 

...

/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ttpfshot_sl done in: 2.9277570645014444 minutes
ttpfshot_sl: {'T1567.002', 'T1560.001', 'T1102.002', 'T1219', 'T1550.002', 'T1548.003', 'T1105', 'T1076', 'T1083', 'T1070.004', 'T1027', 'T1561', 'T1583.001', 'T1562.011', 'T1140', 'T1562.004', 'T1218', 'T1201', 'T1110.004', 'T1555.004', 'T1562.001', 'T1110.001', 'T1518.001', 'T1598.004', 'T1490', 'T1021.001', 'T1537', 'T1588.001', 'T1599', 'T1486', 'T1053.005', 'T1094', 'T1059.003', 'T1553.006', 'T1059', 'T1016', 'T1555.003', 'T1556.002', 'T1657', 'T1547.001', 'T1111', 'T1087', 'T1562.009', 'T1591', 'T1055.001', 'T1048.003', 'T1069.002', 'T1218.005', 'T1136.001', 'T1597.001', 'T1485', 'T1021', 'T1555.005'}
text nr:  1
ttpfshot_sl done in: 1.6850587368011474 minutes
ttpfshot_sl: {'T1090.001', 'T1591.004', 'T1070', 'T1136.003', 'T1586.003', 'T1548.005', 'T1098.005', 'T1531', 'T1110.003', 'T1621', 'T1550.001', 'T1201', 'T1588.002', 'T1071.001', 'T1078.003', 'T1665', 'T1190', 'T1528', 'T1599', 'T1108', 'T1589.002', 'T1029', 'T1111', 'T1078',

/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ttpfshot_sl done in: 3.442954500516256 minutes
ttpfshot_sl: {'T1567.002', 'T1560.001', 'T1102.002', 'T1219', 'T1550.002', 'T1548.003', 'T1105', 'T1076', 'T1083', 'T1070.004', 'T1027', 'T1561', 'T1110.003', 'T1583.001', 'T1562.004', 'T1218', 'T1201', 'T1110', 'T1555.004', 'T1562.001', 'T1110.001', 'T1518.001', 'T1598.004', 'T1490', 'T1021.001', 'T1537', 'T1070.001', 'T1588.001', 'T1599', 'T1486', 'T1053.005', 'T1094', 'T1059.003', 'T1553.006', 'T1059', 'T1016', 'T1555.003', 'T1556.002', 'T1657', 'T1547.001', 'T1111', 'T1087', 'T1562.009', 'T1591', 'T1055.001', 'T1048.003', 'T1069.002', 'T1218.005', 'T1136.001', 'T1597.001', 'T1555.005'}
text nr:  1
ttpfshot_sl done in: 1.5899027307828268 minutes
ttpfshot_sl: {'T1090.001', 'T1591.004', 'T1070', 'T1586.003', 'T1136.003', 'T1548.005', 'T1098.005', 'T1110.003', 'T1621', 'T1550.001', 'T1201', 'T1588.002', 'T1071.001', 'T1078.003', 'T1665', 'T1190', 'T1528', 'T1078.002', 'T1599', 'T1108', 'T1589.002', 'T1029', 'T1111', 'T1078', 'T1090.002', '

/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Simulate different distance tresholds

In [115]:
for min_distance in [0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48]:
    print("min_distance:", min_distance)


    #df_reports["ttpxhunter"] = ""
    df_reports["ttpfshot_sl"] = ""
    #df_reports["ttpfshot_ml"] = ""
    #df_reports["ttpfshot_zs"] = ""

    for index, row in df_reports.iterrows():
        #print(f"Index: {index}")
        print("text nr: ", index)

        text_to_classify = row["CleanText"]


        start_time = time.time()
        ttpfshot_sl = ttf_shot_predict(sent_tokenize(text_to_classify), "SINGLE_LABEL2", llm, db, distance=min_distance)
        elapsed_time = (time.time() - start_time)/60
        print(f"ttpfshot_sl done in: {elapsed_time} minutes")

        '''
        start_time = time.time()
        ttpfshot_ml = ttf_shot_predict(sent_tokenize(text_to_classify), "MULTIPLE_LABEL", llm, db3)
        elapsed_time = (time.time() - start_time)/60
        print(f"ttpfshot_ml done in: {elapsed_time} minutes")

        start_time = time.time()
        ttpfshot_zs = ttf_shot_predict(sent_tokenize(text_to_classify), "ZERO_SHOT", llm, db3)
        elapsed_time = (time.time() - start_time)/60
        print(f"ttpfshot_zs done in: {elapsed_time} minutes")

        start_time = time.time()
        ttps, ttp_names = extract_ttp_from_sentences(sent_tokenize(text_to_classify), th, label_dict, ttpid2name)#(text_to_classify, th, label_dict, ttpid2name)
        elapsed_time = (time.time() - start_time)/60
        print(f"ttpxhunter done in: {elapsed_time} minutes")
        '''
        ### save results to the reports dataframe
        #df_reports.at[index, 'ttpxhunter'] = ttps
        df_reports.at[index, 'ttpfshot_sl'] = ttpfshot_sl
        #df_reports.at[index, 'ttpfshot_ml'] = ttpfshot_ml
        #df_reports.at[index, 'ttpfshot_zs'] = ttpfshot_zs


        #print("ttpxhunter:", ttps)
        #print("ttpxhunter:", ttps)
        print("ttpfshot_sl:", ttpfshot_sl)
        #print("ttpfshot_ml:", ttpfshot_ml)
        #print("ttpZshot_zs:", ttpfshot_zs)
        
    all_scores = {
        #"ttpxhunter" : calculate_metrics2(df_reports, 'TTP', 'ttpxhunter', 'ttpxhunter'),
        "ttpfshot_sl" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_sl', 'ttpfshot_sl'),
        #"ttpfshot_ml" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_ml', 'ttpfshot_ml'),
        #"ttpfshot_zs" : calculate_metrics2(df_reports, 'TTP', 'ttpfshot_zs', 'ttpfshot_zs')
    }

    #scores_df = pd.DataFrame(all_scores)#.transpose()

    #print(scores_df)

    flattened_data = {outer_key: inner_dict[outer_key] for outer_key, inner_dict in all_scores.items()}

    # Create DataFrame
    scores_df = pd.DataFrame(flattened_data).transpose()
    print(scores_df)
        

min_distance: 0.41
text nr:  0
ttpfshot_sl done in: 2.7276632030804953 minutes
ttpfshot_sl: {'T1567.002', 'T1560.001', 'T1548.003', 'T1219', 'T1550.002', 'T1105', 'T0869', 'T1076', 'T1083', 'T1070.004', 'T1027', 'T1561', 'T1562.004', 'T1556', 'T1201', 'T1588.002', 'T1588', 'T1562.001', 'T1110.001', 'T1518.001', 'T1587', 'T1598.004', 'T1480.001', 'T1490', 'T1537', 'T1553', 'T1070.001', 'T1560', 'T1599', 'T1486', 'T1053.005', 'T1530', 'T1094', 'T1059.003', 'T1553.006', 'T1059', 'T1016', 'T1555.003', 'T1657', 'T1547.001', 'T1018', 'T1071.003', 'T1003.005', 'T1556.004', 'T1562.009', 'T1591', 'T1055.001', 'T1048.003', 'T1069.002', 'T1218.005', 'T1136.001', 'T1021', 'T1555.005'}
text nr:  1
ttpfshot_sl done in: 1.4641175071398418 minutes
ttpfshot_sl: {'T1591.004', 'T1210', 'T1599.001', 'T1548.005', 'T1583.003', 'T1098.005', 'T1110.003', 'T1562.011', 'T1548', 'T1621', 'T1201', 'T1588.002', 'T1071.001', 'T1078.003', 'T1193', 'T1665', 'T1190', 'T1528', 'T1108', 'T1589.002', 'T1029', 'T1189', 'T

/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ttpfshot_sl done in: 2.940241769949595 minutes
ttpfshot_sl: {'T1567.002', 'T1070', 'T1219', 'T1550.002', 'T1105', 'T1592.002', 'T0869', 'T1076', 'T1083', 'T1070.004', 'T1027', 'T1561', 'T1136.002', 'T1562.004', 'T1218', 'T1201', 'T1588.002', 'T1555.004', 'T1562.001', 'T1110.001', 'T1548.002', 'T1518.001', 'T1587', 'T1598.004', 'T1480.001', 'T1490', 'T1537', 'T1070.001', 'T1190', 'T1560', 'T1599', 'T1486', 'T1053.005', 'T1094', 'T1059.003', 'T1553.006', 'T1059', 'T1016', 'T1555.003', 'T1657', 'T1547.001', 'T1071.003', 'T1556.004', 'T1562.009', 'T1591', 'T1055.001', 'T1048.003', 'T1069.002', 'T1218.005', 'T1136.001', 'T1555.005'}
text nr:  1
ttpfshot_sl done in: 1.3485962788263957 minutes
ttpfshot_sl: {'T1591.004', 'T1210', 'T1599.001', 'T1136.003', 'T1098.005', 'T1110.003', 'T1562.011', 'T1200', 'T1621', 'T1201', 'T1550.001', 'T1588.002', 'T1071.001', 'T1078.003', 'T1193', 'T1665', 'T1528', 'T1108', 'T1589.002', 'T1029', 'T1189', 'T1078.001', 'T1111', 'T1562.008', 'T1078', 'T1090.002', 

/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ttpfshot_sl done in: 2.5813935955365497 minutes
ttpfshot_sl: {'T1567.002', 'T1560.001', 'T1102.002', 'T1219', 'T1550.002', 'T1548.003', 'T1105', 'T1610', 'T1076', 'T1083', 'T1589', 'T1070.004', 'T1027', 'T1556.009', 'T1561', 'T1556.006', 'T1140', 'T1562.004', 'T1218', 'T1201', 'T1588.002', 'T1555.004', 'T1562.001', 'T1110.001', 'T1518.001', 'T1587', 'T1598.004', 'T1480.001', 'T1490', 'T1537', 'T1070.001', 'T1190', 'T1599', 'T1486', 'T1053.005', 'T1094', 'T1059.003', 'T1553.006', 'T1059', 'T1016', 'T1555.003', 'T1529', 'T1657', 'T1547.001', 'T1562.009', 'T1591', 'T1055.001', 'T1048.003', 'T1069.002', 'T1218.005', 'T1136.001', 'T1555.005'}
text nr:  1
ttpfshot_sl done in: 1.2834811409314473 minutes
ttpfshot_sl: {'T1591.004', 'T1210', 'T1599.001', 'T1136.003', 'T1098.005', 'T1580', 'T1110.003', 'T1562.011', 'T1090', 'T1200', 'T1021.008', 'T1621', 'T1201', 'T1110', 'T1588.002', 'T1071.001', 'T1078.003', 'T1665', 'T1528', 'T1190', 'T1649', 'T1108', 'T1589.002', 'T1195', 'T1029', 'T1189', 'T

...

ttpfshot_sl done in: 2.7713734189669292 minutes
ttpfshot_sl: {'T1567.002', 'T1102.002', 'T1219', 'T1550.002', 'T1548.003', 'T1105', 'T1076', 'T1083', 'T1070.004', 'T1027', 'T1110.003', 'T1562.004', 'T1218', 'T1201', 'T1098', 'T1597.002', 'T1588', 'T1562.001', 'T1110.001', 'T1518.001', 'T1587', 'T1598.004', 'T1563', 'T1490', 'T1021.001', 'T1537', 'T1588.001', 'T1560', 'T1599', 'T1486', 'T1053.005', 'T1094', 'T1059.003', 'T1553.006', 'T1059', 'T1016', 'T1555.003', 'T1657', 'T1547.001', 'T1003.005', 'T1087', 'T1556.004', 'T1562.009', 'T1591', 'T1218.005', 'T1048.003', 'T1069.002', 'T1136.001', 'T1555.005'}
text nr:  1
ttpfshot_sl done in: 1.6152965784072877 minutes
ttpfshot_sl: {'T1548.003', 'T1070', 'T1136.003', 'T1498', 'T1098.005', 'T1110.003', 'T1090', 'T1021.008', 'T1621', 'T1201', 'T1588.002', 'T1071.001', 'T1078.003', 'T1193', 'T1665', 'T1190', 'T1528', 'T1078.002', 'T1599', 'T1649', 'T1108', 'T1589.002', 'T1029', 'T1657', 'T1111', 'T1078', 'T1090.002', 'T1036', 'T1098.001', 'T1195

/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ttpfshot_sl done in: 2.8237418214480083 minutes
ttpfshot_sl: {'T1567.002', 'T1560.001', 'T1102.002', 'T1219', 'T1550.002', 'T1548.003', 'T1105', 'T0869', 'T1076', 'T1083', 'T1070.004', 'T1027', 'T1561', 'T1110.003', 'T1562.004', 'T1218', 'T1556', 'T1201', 'T1110.004', 'T1597.002', 'T1588', 'T1562.001', 'T1110.001', 'T1548.002', 'T1518.001', 'T1587', 'T1598.004', 'T1563', 'T1490', 'T1021.001', 'T1537', 'T1588.001', 'T1599', 'T1486', 'T1094', 'T1059.003', 'T1553.006', 'T1059', 'T1016', 'T1555.003', 'T1657', 'T1547.001', 'T1111', 'T1003.005', 'T1087', 'T1562.009', 'T1591', 'T1055.001', 'T1048.003', 'T1069.002', 'T1218.005', 'T1136.001', 'T1555.005'}
text nr:  1
ttpfshot_sl done in: 1.374034051100413 minutes
ttpfshot_sl: {'T1591.004', 'T1548.003', 'T1070', 'T1136.003', 'T1498', 'T1098.005', 'T1110.003', 'T1090', 'T1021.008', 'T1621', 'T1201', 'T1588.002', 'T1071.001', 'T1078.003', 'T1193', 'T1665', 'T1190', 'T1528', 'T1078.002', 'T1599', 'T1649', 'T1108', 'T1589.002', 'T1029', 'T1078', 'T1

/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hamzicd/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}